<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 13. Interfaces, Protocols, and ABCs

Program to an interface, not an implementation

Since python 3.8, we have four ways to define an interfaces:
 - Duck typing: Python's default approach
 - Goose typing: supported by abstract base classes (ABCs) which rely on runtime checks of objects against ABCs.
 - Static typing: traditional approach of statically-typed langauges like C and Java
 - Static duck typing: Go langauge approach; supported by subclasses of `typing.Protocol`

## Two Kinds of Protocols

An object protocol specifies methods which an object must provide to fulfill a role.

In [1]:
class Vowels:
  def __getitem__(self, i):
    return 'AEIOU'[i]

In [2]:
v = Vowels()
v[0]

'A'

In [3]:
v[-1]

'U'

In [4]:
for c in v:
  print(c)

A
E
I
O
U


In [5]:
'E' in v

True

In [6]:
'Z' in v

False

Implementing `__getitem__` is enough to allow retrieving items by index, and also to support iteration and the `in` operator.

## Programming Ducks

Sequence and iterable protocols

In summary, given the importance of sequence-like data structures, Python manages to make iteration and the `in` operator work by invoking `__getitem__` when `__iter__` and `__contains` are unavailable

In [8]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

The original `FrenchDeck` does not subclass `abc.Sequence` either, but it does implement both methods of the sequence protocol: `__getitem__` and `__len__`

### Why static type checkers have no chance of dealing with them?

## Monkey Patching: Implementing a Protocol at Runtime

In [9]:
from random import shuffle

l = list(range(10))
shuffle(l)

In [10]:
l

[7, 5, 9, 0, 1, 4, 2, 8, 3, 6]

In [16]:
deck = FrenchDeck()

In [17]:
shuffle(deck)

In [13]:
def set_card(deck, position, card):
  deck._cards[position] = card

In [14]:
FrenchDeck.__setitem__ = set_card

In [18]:
shuffle(deck)

In [19]:
deck[:5]

[Card(rank='2', suit='hearts'),
 Card(rank='6', suit='hearts'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='J', suit='clubs'),
 Card(rank='A', suit='hearts')]

## Defensive Programming and "Fail Fast"

In [ ]:
def __init__(self, iterable):
  # don't type check iterable is iterable
  # just use list to see whether iterable is iterable
  self._balls = list(iterable)


## Goose Typing

In [20]:
class Struggle:
  def __len__(self): return 23

In [21]:
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [22]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

# not check for the implementation of the abstract methods at import time
# but only at runtime when we actually try to instantiate FrenchDeck2
class FrenchDeck2(abc.MutableSequence):
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

  def __setitem__(self, position, value):
    self._cards[position] = value

  def __delitem__(self, position):
    del self._cards[position]

  def insert(self, position, value):
    self._cards.insert(position, value)